In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
def get_data_from_dir(dir):
    tweet_file = [dir + file for file in os.listdir(dir) if 'zip' in file]
    return tweet_file

In [25]:
tweet_list = get_data_from_dir('../AllData/dataset/')
tweet_list.pop(5)
tweet_list.pop(6)
tweet_list

['../AllData/dataset/AI_tweets.csv.zip',
 '../AllData/dataset/bitcoin_tweets.csv.zip',
 '../AllData/dataset/crypto_tweets.csv.zip',
 '../AllData/dataset/cyptocurrency_tweets.csv.zip',
 '../AllData/dataset/gpu_tweets.csv.zip',
 '../AllData/dataset/Nvidia_tweets.csv.zip',
 '../AllData/dataset/RTX_tweets.csv.zip']

In [26]:
data = pd.read_csv(tweet_list[5], compression='zip', lineterminator='\n')

In [27]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf

In [ ]:
model = tf.keras.models.load_model('../../../model_v2')
